In [ ]:
!pip install yfinance
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=01fb5a5a87136d09eeea4e27af77877116bc46a085348b3564640e00c65af82c
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
Stock = "RELIANCE.NS"
data = yf.download(Stock, start="2016-01-01", end="2020-10-31")

[*********************100%***********************]  1 of 1 completed


In [ ]:
T3 = pd.DataFrame({"Close": data["Close"]})
T3['Year'] = T3.index.year 
T2 = pd.DataFrame({"cumpnl_long":['0'], "SMA":['0'],  "LMA":['0'], "Year":['0']})

for z in range (2016, 2021, 1):
  T = T3.where(T3.Year == z)
  T = T.dropna()
  for a in range(1,35,2):
    for b in range(a,35,2):
      SMA=a
      LMA=b

      # Compute Moving averages of last 10 days and 30 days closing prices
      T['Short_average'] = T['Close'].rolling(window=SMA, min_periods=1, center=False).mean()
      T['Long_average'] = T['Close'].rolling(window=LMA, min_periods=1, center=False).mean()

      T['positions_long'] = np.nan

      for x in range (len(T)):
        if T.Short_average[x] > T.Long_average[x]:
          T['positions_long'][x] = 1
        if T.Short_average[x] <= T.Long_average[x]:
          T['positions_long'][x] = 0

      T.positions_long = T.positions_long.fillna(method='ffill')  

      T['price_difference']= T.Close - T.Close.shift(1)
      T['pnllong'] = T.positions_long.shift(1) * T.price_difference
      T['cumpnl_long'] = T.pnllong.cumsum()
      T1 = T[['cumpnl_long']].tail(1)
      T1['SMA'] = SMA
      T1['LMA'] = LMA
      T1['Year'] = z
      T2 = T2.append(T1)


print(T2)

                    cumpnl_long SMA LMA  Year
0                             0   0   0     0
2016-12-30 00:00:00           0   1   1  2016
2016-12-30 00:00:00    -55.6749   1   3  2016
2016-12-30 00:00:00    -9.02502   1   5  2016
2016-12-30 00:00:00     4.82504   1   7  2016
...                         ...  ..  ..   ...
2020-10-30 00:00:00      619.95  29  31  2020
2020-10-30 00:00:00       370.8  29  33  2020
2020-10-30 00:00:00           0  31  31  2020
2020-10-30 00:00:00      399.45  31  33  2020
2020-10-30 00:00:00           0  33  33  2020

[766 rows x 4 columns]


In [ ]:
Pivot_Table1 = pd.pivot_table(T2, values ='cumpnl_long', index =['SMA', 'LMA'], columns =['Year'], aggfunc = np.sum) 
print (Pivot_Table1)

Year        2016     2017    2018    2019     2020    0
SMA LMA                                                
1   1          0        0       0       0        0  NaN
    3   -55.6749  432.525   382.5   240.9  323.269  NaN
    5   -9.02502   353.75  367.45   361.8  326.299  NaN
    7    4.82504   330.95    77.7  405.75   457.61  NaN
    9     -33.35    306.7   101.8     283   576.56  NaN
...          ...      ...     ...     ...      ...  ...
29  33  -3.72504  207.625  131.65   221.9    370.8  NaN
31  31         0        0       0       0        0  NaN
    33   4.27499   262.65   163.1  208.05   399.45  NaN
33  33         0        0       0       0        0  NaN
0   0        NaN      NaN     NaN     NaN      NaN    0

[154 rows x 6 columns]


In [ ]:
Pivot_Table1.to_csv("PV_T.csv", index=True, encoding='utf8')
from google.colab import files
files.download('PV_T.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>